In [1]:
import requests
import json
import time
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell

    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data).text)                   
    end = time.time()                                                              
    print("Took: %s ms" % (end - start))

    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
    elif 'error-message' in response:
        return print(response['error-message'])
    else:
        return print(response)

In [2]:
server='http://localhost:9090/jsoniq'

In [3]:
%%rumble
1+1

Took: 0.3440520763397217 ms
2


## ML pipelines

In [ ]:
{"id": 0, "label": 1, "pixel_00": 0.2, "pixel_01": 0.3, ... , "pixel_2828": 0.32}

In [4]:
%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};
let $training-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
    }
let $test-data := validate type local:mytype* {
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
    }
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "pca_1_output", "k": 2})
let $pca2 := get-estimator("PCA", {"inputCol": "features", "outputCol": "pca_2_output", "k": 3})
let $vector-assembler_2 := get-transformer("VectorAssembler", {"inputCols" : ["pca_1_output","pca_2_output"], "outputCol" : "transformedFeatures"})
let $naivebayes := get-estimator("NaiveBayes", {"featuresCol": "transformedFeatures"})
let $pipeline := get-estimator("Pipeline", {"stages": [$vector-assembler, $pca, $pca2, $vector-assembler_2, $naivebayes]})
let $pip := $pipeline($training-data, {})
return $pip($test-data, {})

Took: 8.283988952636719 ms
It seems that you are not using Java 8. Spark only works with Java 8. If you have several versions of java installed, you need to set your JAVA_HOME accordingly. If you do not have Java 8 installed, we recommend installing AdoptOpenJDK 1.8.


In [5]:
%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};

let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "features2", "k": 2})
let $logisticregression := get-estimator("LogisticRegression", {"featuresCol": "features2"})
let $pipeline := get-estimator("Pipeline", {"stages": [$vector-assembler, $pca, $logisticregression]})

let $training-data := validate type local:mytype* {
  {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
  {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
  {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
  {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
}
let $test-data := validate type local:mytype* {
  {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
  {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
  {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
}
let $pip := $pipeline($training-data, {})
return $pip($test-data, {})

Took: 1.6907620429992676 ms
{"id": 0, "label": 1, "col1": -1, "col2": 1.5, "col3": 1.3, "features": [-1, 1.5, 1.3], "features2": [0.6811443247067491, -1.657406267583813], "rawPrediction": [-37.0195404210944, 37.0195404210944], "probability": [8.367926801087957e-17, 0.9999999999999999], "prediction": 1}
{"id": 1, "label": 0, "col1": 3, "col2": 2, "col3": -0.1, "features": [3, 2, -0.1], "features2": [-2.93898570550277, 0.4844169482619757], "rawPrediction": [37.84452856407826, -37.84452856407826], "probability": [1, 0], "prediction": 0}
{"id": 2, "label": 1, "col1": 0, "col2": 2.2, "col3": -1.5, "features": [0, 2.2, -1.5], "features2": [0.2860939487717081, 1.1822492465709793], "rawPrediction": [-18.53660987325421, 18.53660987325421], "probability": [8.905383262039802e-09, 0.9999999910946168], "prediction": 1}


%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"};
let $training-data := validate type local:mytype* {
  {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
  {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
  {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
  {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
}
let $test-data := validate type local:mytype* {
  {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
  {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
  {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
}
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col1", "col2", "col3" ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "features2", "k": 2})
let $logisticregression := get-estimator("LogisticRegression", {"featuresCol": "features

In [83]:
%%rumble
declare type local:mytype as {"id": "integer", "label": "integer", "col_0": "decimal", "col_1": "decimal", "col_2": "decimal", "col_3": "decimal", "col_4": "decimal", "col_5": "decimal", "col_6": "decimal", "col_7": "decimal", "col_8": "decimal", "col_9": "decimal", "col_10": "decimal", "col_11": "decimal", "col_12": "decimal", "col_13": "decimal"};

let $training-data := validate type local:mytype* {
    csv-file("")
}
let $test-data := validate type local:mytype* {
  json-file("https://gist.githubusercontent.com/daviddao/31652ddf8226143688ae3fe1008c0bb7/raw/15a9b0f9c005e5b42d575ef4701c8146cfc31bab/X_train.jsonl")
}
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ "col_0", "col_1", "col_2", "col_3", "col_4", "col_5", "col_6", "col_7", "col_8", "col_9", "col_10", "col_11", "col_12", "col_13" ], 
                                                             "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", 
                                  "outputCol": "features2", 
                                  "k": 2})
let $logisticregression := get-estimator("LogisticRegression", {"maxIter" : 1000 
                                                                ,"featuresCol": "features2"})
let $pipeline := get-estimator("Pipeline", {"stages": [$vector-assembler, $pca, $logisticregression]})
let $pip := $pipeline($training-data, {})
return $pip($test-data, {})

Took: 3.4312150478363037 ms
"Warning! The output sequence contains 1592 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"id": 0, "label": 1, "col_0": 50, "col_1": 0, "col_2": 23686, "col_3": 1, "col_4": 10, "col_5": 0, "col_6": 8, "col_7": 4, "col_8": 0, "col_9": 0, "col_10": 0, "col_11": 0, "col_12": 35, "col_13": 0, "features": {"0": 50, "2": 23686, "3": 1, "4": 10, "6": 8, "7": 4, "12": 35}, "features2": [23685.99872709563, 3.707822042745316], "rawPrediction": [0.8833110819978184, -0.8833110819978184], "probability": [0.7075078884579349, 0.2924921115420651], "prediction": 0}
{"id": 1, "label": 1, "col_0": 47, "col_1": 0, "col_2": 158451, "col_3": 10, "col_4": 14, "col_5": 0, "col_6": 5, "col_7": 0, "col_8": 0, "col_9": 0, "col_10": 0, "col_11": 0, "col_12": 2, "col_13": 0, "features": {"0": 47, "2": 158451, "3": 10, "4": 14, "6": 5, "12": 2}, "features2": [158450.99421703297

## Previous explorations

In [6]:
%%rumble
let $training-data := (
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
)
    
let $test-data := (
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5},
)

let $my-training-data := annotate($training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"})
let $my-test-data := annotate($test-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"})

let $est := get-estimator("KMeans")
let $tra := $est(
    $my-training-data,
    {"featuresCol": ["col1", "col2", "col3"]}
)

for $i in $tra(
    $my-test-data,
    {"featuresCol": ["col1", "col2", "col3"]}
)
return $i

Took: 0.02805805206298828 ms
There was an error on line 28 in file:/Users/david/Projects/rumble/:


Code: [XPST0003]
Message: Parser failed. 

Metadata: file:/Users/david/Projects/rumble/:LINE:28:COLUMN:0:
This code can also be looked up in the documentation and specifications for more information.



In [7]:
%%rumble
let $training-data := (
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
)
    
let $test-data := (
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
)

let $my-training-data := annotate($training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"})
let $my-test-data := annotate($test-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"})

let $est := get-estimator("PCA")
let $trained_est := $est($my-training-data,
                 {"inputCol": ["col1", "col2", "col3"], "outputCol": "ocol1", "k":2})

let $new-training-data := $trained_est($my-training-data, {"inputCol": ["col1", "col2", "col3"], "outputCol": "ocol1"})

let $my-new-training-data := annotate($new-training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal", "ocol1": ["decimal"]})
    
let $est2 := get-estimator("LogisticRegression")
let $trained_est2 := $est2($my-new-training-data, {"inputCol": ["ocol1", "ocol2", "ocol3"]})
    
for $i in $trained_est2(
    $my-test-data,
    {"inputCol": ["col1"], "outputCol": ["ocol1"]}
)
return $i



Took: 4.359323978424072 ms
There was an error on line 18 in file:/Users/david/Projects/rumble/:

let $trained_est := $est($my-training-data,
                    ^

Code: [RBML0003]
Message: Invalid Param; There is an issue with the parameters provided to the estimator PCA.
It seems you provided an array of strings ([ "col1", "col2", "col3" ]) for parameter featuresCol, inputCol or similar.
However, this parameter should be a string, which is the name of the field associated with an array of features to train on or to transform.
If you do not have such a field in your data, then you can build it with the VectorAssembler transformer by combining the fields you want to include.

For example:
let $vector-assembler := get-transformer("VectorAssembler")
let $data-with-features := $vector-assembler($data, {"inputCols" : [ "age", "weight" ], "outputCol" : "features" })

and then
let $est := get-estimator("PCA")
let $model := $est($data-with-features, {"featuresCol" : "features" }) (: assuming 

In [ ]:
let $est := get-estimator("LogisticRegression")
let $tra := $est($scaled-data,
                 {"featuresCol": ["col1", "col2", "col3"], "labelCol": "label"})

for $i in $tra(
    $my-test-data,
    {"featuresCol": ["col1", "col2", "col3"]}
)

In [106]:
%%rumble
declare function local:pipeline($trainingset, $testset)
{
  let $logisticregression := get-estimator("LogisticRegression")
  let $model := $logisticregression($trainingset, {"featuresCol": ["col1", "col2", "col3"]})
  return $model($testset, {"featuresCol": ["col1", "col2", "col3"]})
};

let $training-data := (
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
)
    
let $test-data := (
    {"id": 0, "label": 1, "col1": -1.0, "col2": 1.5, "col3": 1.3},
    {"id": 1, "label": 0, "col1": 3.0, "col2": 2.0, "col3": -0.1},
    {"id": 2, "label": 1, "col1": 0.0, "col2": 2.2, "col3": -1.5}
)

let $my-training-data := annotate($training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"}) 
let $my-test-data := annotate($test-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"}) 
    
let $pipeline-model := local:pipeline($my-training-data, ?)
return $pipeline-model($my-test-data) 


Took: 0.029755830764770508 ms


'There was an error.\n\nCode: [XPST0003] (this code can be looked up in the documentation and specifications).\n\nLocation information: file:/Users/david/Projects/scripts/:LINE:26:COLUMN:0:\n\nParser failed. \n'

In [ ]:
%%rumble
declare function local:pipeline($trainingset, $testset)
{
  let $logisticregression := get-estimator("LogisticRegression")
  let $model := $logisticregression($trainingset, {"featuresCol": ["col1", "col2", "col3"]})
  return $model($testset, {"featuresCol": ["col1", "col2", "col3"]})
};

let $training-data := json-file("/Users/David/Desktop/Rumble/X_train.jsonl")
    
let $test-data := json-file("")

let $my-training-data := annotate($training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"}) 
let $my-test-data := annotate($test-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"}) 
    
let $pipeline-model := local:pipeline($my-training-data, ?)
return $pipeline-model($my-test-data) 

In [121]:
%%rumble
import module namespace m = "https://gist.githubusercontent.com/daviddao/0ed40973c244398120741edd7c2cb657/raw/562ac253b86b36c39aea21efa93fca59a2d97b60/rumble_3.jq";

m:my_function(1,2,3),
m:my_other_function(1,2,3),
let $est := get-estimator("PCA")
    
let $training-data := (
    {"id": 0, "label": 1, "col1": 0.0, "col2": 1.1, "col3": 0.1},
    {"id": 1, "label": 0, "col1": 2.0, "col2": 1.0, "col3": -1.0},
    {"id": 2, "label": 0, "col1": 2.0, "col2": 1.3, "col3": 1.0},
    {"id": 3, "label": 1, "col1": 0.0, "col2": 1.2, "col3": -0.5}
)
    
let $my-training-data := annotate($training-data, {"id": "integer", "label": "integer", "col1": "decimal", "col2": "decimal", "col3": "decimal"}) 

    
return $est => m:fit_transform($my-training-data, {"inputCol": ["col1", "col2", "col3"], "k": 2})


Took: 0.582643985748291 ms


"There was an error.\n\nCode: [RBML0003] (this code can be looked up in the documentation and specifications).\n\nLocation information: file:/Users/david/Projects/scripts/:LINE:5:COLUMN:12:\n\nInvalid Param; Parameters provided to PCAModel causes the following error: Missing parameter value for 'inputCol'."

In [115]:
%%rumble
module namespace my_module = "rumble_3.jq";

declare function my_module:my_function($a, $b, $c) {
    $a + $b + $c
};

declare function my_module:my_other_function($a, $b, $c) {
    $a + $b + $c + 1
};

declare variable $my_module:d := 5;

declare function my_module:fit_transform($est, $X_train, $params) {
    let $trained_est := $est($X_train, $params)
    return $trained_est($X_train, $params)
};

Took: 0.014400005340576172 ms


'There was an error.\n\nCode: [XPST0003] (this code can be looked up in the documentation and specifications).\n\nLocation information: none:LINE:0:COLUMN:0:\n\nParser failed. A library module is not executable.'

## Convert data

In [11]:
import numpy as np
data = np.load("/Users/david/Desktop/Rumble/data.npz")
X_train = data['X']
y_train = data['y']
X_test = data['X_test'] 
y_test = data['y_test']

In [43]:
ls = []
for i,j in zip(col_names, X_train[0]):
    ls += [f"'{i}': {j}"]
",".join(ls)

"'col_0': 50.0,'col_1': 0.0,'col_2': 23686.0,'col_3': 1.0,'col_4': 10.0,'col_5': 0.0,'col_6': 8.0,'col_7': 4.0,'col_8': 0.0,'col_9': 0.0,'col_10': 0.0,'col_11': 0.0,'col_12': 35.0,'col_13': 0.0"

In [71]:
def create_dataset(X, y, file_name, col_names=None):
    
    # create col names
    if col_names is None:
        col_names = [f'col_{i}' for i in range(X.shape[1])]
    else:
        col_names = col_names
    
    # loop through dataset
    data_ls = []
    for ids, labels in enumerate(y):
        label = y[ids]
        # create dataset columns
        ls = []
        for i,j in zip(col_names, X_train[ids]):
            ls += [f"'{i}': {j}"]
        cols_features = ",".join(ls)
        row = f"{{'id': {ids}, 'label': {label}, {cols_features}}}"
        data_ls += [row]
    # save as json
    res = "\n".join(data_ls)
    
    dataset = (
    f'validate type local:mytype* {{\n'
    f'{res}\n'
    f'}}\n'
    )

    text_file = open(f"/Users/david/Desktop/Rumble/{file_name}.json", "w")
    n = text_file.write(res)
    text_file.close()
    
create_dataset(X_train, y_train, "X_train")
create_dataset(X_test, y_test, "X_test")
    
    
def create_datasets_in_rumble(X_train, y_train, X_test, y_test, col_names=None):
    
    # create col names
    if col_names is None:
        col_names = [f'col_{i}' for i in range(X_train.shape[1])]
    else:
        col_names = col_names
        
    col_names_string = ', '.join(col_names)
    col_string = [f" {i}: decimal" for i in col_names]
    col_string = ','.join(col_string)
    data_type = f'declare type local:mytype as {{"id": "integer", "label": "integer", {col_string}}};'
    
    tr_data = create_dataset(X_train, y_train, "X_train")
    test_data = create_dataset(X_test, y_test, "X_test")
        
    # defining operators
    vector = ('$vector-assembler',f'get-transformer("VectorAssembler", {{"inputCols" : [ {col_names_string} ], "outputCol" : "features"}})')
    pca = ('$pca','get-estimator("PCA", {"inputCol": "features", "outputCol": "features2", "k": 2})')
    logreg = ('$logisticregression','get-estimator("LogisticRegression", {"featuresCol": "features2"})')

    ops = [vector, pca, logreg]

    def extract_definitions(ops):
        res = ''
        for op in ops:
            res = res + f'let {op[0]} := {op[1]}\n'
        return res

    def create_pipeline(ops):
        arr = '['
        for op in ops[:-1]:
            arr += f'{op[0]}, '
        arr += f'{ops[-1][0]}]'
        res = f'get-estimator("Pipeline", {{"stages": {arr}}})'
        return res

    definitions = extract_definitions(ops)
    pipeline = create_pipeline(ops)
    call_pipeline = 'let $pip := $pipeline($training-data, {})'

    program = (
        f'%%rumble\n'
        f'{data_type}\n'
        f'let $training-data := {tr_data}\n'
        f'let $test-data := {test_data}\n'
        f'{definitions}'
        f'let $pipeline := {pipeline}\n'
        f'let $pip := $pipeline($training-data, {{}})\n'
        f'return $pip($test-data, {{}})'
    )

    return program

rumble_program = create_datasets_in_rumble(X_train, y_train, X_test, y_test)


print(rumble_program)


%%rumble
declare type local:mytype as {"id": "integer", "label": "integer",  col_0: decimal, col_1: decimal, col_2: decimal, col_3: decimal, col_4: decimal, col_5: decimal, col_6: decimal, col_7: decimal, col_8: decimal, col_9: decimal, col_10: decimal, col_11: decimal, col_12: decimal, col_13: decimal};
let $training-data := None
let $test-data := None
let $vector-assembler := get-transformer("VectorAssembler", {"inputCols" : [ col_0, col_1, col_2, col_3, col_4, col_5, col_6, col_7, col_8, col_9, col_10, col_11, col_12, col_13 ], "outputCol" : "features"})
let $pca := get-estimator("PCA", {"inputCol": "features", "outputCol": "features2", "k": 2})
let $logisticregression := get-estimator("LogisticRegression", {"featuresCol": "features2"})
let $pipeline := get-estimator("Pipeline", {"stages": [$vector-assembler, $pca, $logisticregression]})
let $pip := $pipeline($training-data, {})
return $pip($test-data, {})


In [76]:
%%rumble
for $i in json-file("/Users/david/Desktop/Rumble/X_train.jsonl", 10)
return $i

Took: 0.5156161785125732 ms
"Warning! The output sequence contains 1592 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"id": 0, "label": 1, "col_0": 50, "col_1": 0, "col_2": 23686, "col_3": 1, "col_4": 10, "col_5": 0, "col_6": 8, "col_7": 4, "col_8": 0, "col_9": 0, "col_10": 0, "col_11": 0, "col_12": 35, "col_13": 0}
{"id": 1, "label": 1, "col_0": 47, "col_1": 0, "col_2": 158451, "col_3": 10, "col_4": 14, "col_5": 0, "col_6": 5, "col_7": 0, "col_8": 0, "col_9": 0, "col_10": 0, "col_11": 0, "col_12": 2, "col_13": 0}
{"id": 2, "label": 1, "col_0": 40, "col_1": 4, "col_2": 197012, "col_3": 0, "col_4": 13, "col_5": 1, "col_6": 0, "col_7": 3, "col_8": 0, "col_9": 0, "col_10": 8614, "col_11": 0, "col_12": 40, "col_13": 2}
{"id": 3, "label": 1, "col_0": 30, "col_1": 0, "col_2": 169583, "col_3": 0, "col_4": 13, "col_5": 6, "col_6": 4, "col_7": 0, "col_8": 0, "col_9": 0, "col_10": 0, "